In [35]:
import pandas as pd
import numpy as np

#read database file used in 7-Dedup.
final = pd.read_csv("inputs/deduplicated_PC.csv", low_memory=False)

#read dedupe file
#df = pd.read_csv("outputs/probable_duplicates_CSD.csv")
df = pd.read_csv("outputs/pairs_CSD.csv")


def hierarchy(id1, id2): 

    try:

        check_beds1 = final.loc[final.idx == id1].number_beds.item()
        check_beds2 = final.loc[final.idx == id2].number_beds.item()
        check_type1 = final.loc[final.idx == id1].facility_type.item()
        check_type2 = final.loc[final.idx == id2].facility_type.item()
        check_file1 = final.loc[final.idx == id1].filename.item()
        check_file2 = final.loc[final.idx == id2].filename.item()
        check_name1 = final.loc[final.idx == id1].facility_name.item()
        check_name2 = final.loc[final.idx == id2].facility_name.item()

        filename_priority = ['QC_healthcare_facilities.csv', 'QC_Gatineau_public_places_including_hospitals,csv', 'QC_Quebec_City_public_places.csv', 
                             'ON_Ottawa_hospital.csv', 'ON_Durham_healthcare_facilities.csv', 'ON_healthcare_facilities.csv', 'NB_hospital_horizon.csv', 'NB_nursing_homes.csv']

        facilityname_priority = ['hospital', 'hôpita','hopita']

        if filename_priority.count(check_file1) == 1:

            if filename_priority.count(check_file2) == 1:

                priority1 = filename_priority.index(check_file1)
                priority2 = filename_priority.index(check_file2)

                if priority1 > priority2:
                    return id1
                elif priority1 < priority2:
                    return id2
                elif priority1 == priority2:

                    if any(substring in check_name1.lower() for substring in facilityname_priority):
                        return id2
                    elif any(substring in check_name2.lower() for substring in facilityname_priority):
                        return id1
                    else:
                        return id2

            elif filename_priority.count(check_file2) == 0:
                return id2

        elif filename_priority.count(check_file1) == 0:

            if filename_priority.count(check_file2) == 1:
                return id1

            elif filename_priority.count(check_file2) == 0:

                if check_file1 == 'CIHI_healthcare_facilities.csv' and check_file2 != 'CIHI_healthcare_facilities.csv':
                    return id1

                elif check_file2 == 'CIHI_healthcare_facilities.csv' and check_file1 != 'CIHI_healthcare_facilities.csv':
                    return id2

                else:

                    if check_beds1 == np.nan and check_beds2 == np.nan:

                        if any(substring in check_name1.lower() for substring in facilityname_priority):
                            return id2
                        elif any(substring in check_name2.lower() for substring in facilityname_priority):
                            return id1
                        else:
                            return id2

                    elif check_beds1 == np.nan and check_beds2 != np.nan:
                        return id1

                    elif check_beds1 != np.nan and check_beds2 == np.nan:
                        return id2

                    elif check_beds1 != np.nan and check_beds2 != np.nan:

                        if any(substring in check_name1.lower() for substring in facilityname_priority):
                            return id2
                        elif any(substring in check_name2.lower() for substring in facilityname_priority):
                            return id1
                        else:
                            return id2



    except ValueError:
        return "Item not found."


df["to_remove"] = df.apply(lambda x: hierarchy( id1 = x.idx1, id2 = x.idx2), axis = 1)
#df.to_csv('test.csv')


In [1]:

import pandas as pd
import numpy as np

#read database file used in 7-Dedup.
final = pd.read_csv("inputs/deduplicated_PC.csv", low_memory=False)

#read dedupe file
#df = pd.read_csv("outputs/probable_duplicates_CSD.csv")
df = pd.read_csv("outputs/pairs_CSD.csv")


def check_1(address, distance, name, streetnumber):

    if distance != None:
        if address >= 0.75 and distance < 2 and name > 0.7 and streetnumber == 1:
            return True
        else:
            return False

    else:
        return False

df["Check_1"] = df.apply(lambda x: check_1(address = x.Addr_CS, 
                          distance = x.Distance,
                          name = x.Name_CS,
                          streetnumber = x.StrNum_Match), axis=1)

#df.to_csv('test.csv')

In [2]:
def check_2(address, distance, name):

    if distance != None:
        if address >= 0.75 and distance < 1 and name > 0.85:
            return True
        else:
            return False

    else:
        return False

df["Check_2"] = df.apply(lambda x: check_2(address = x.Addr_CS, 
                          distance = x.Distance,
                          name = x.Name_CS), axis=1)


In [3]:
def check_3(address, distance, file):

    if distance != None:
        if address >= 0.99 and distance <= 0.5 and file == 1:
            return True
        else:
            return False

    else:
        return False

df["Check_3"] = df.apply(lambda x: check_3(address = x.Addr_CS, 
                          distance = x.Distance,
                          file = x.File_Match), axis=1)

In [4]:
def check_4(address, distance, streetnumber):


    if distance != None:
        if address >= 0.8 and distance < 0.1 and streetnumber == 1:
            return True
        else:
            return False

    else:
        return False

df["Check_4"] = df.apply(lambda x: check_4(address = x.Addr_CS, 
                          distance = x.Distance, 
                          streetnumber = x.StrNum_Match), axis=1)

In [5]:
def check_5(address, distance, name, streetnumber):

    if distance != None:
        if address >= 0.90 and distance < 2 and name > 0.93 and streetnumber == 1:
            return True
        else:
            return False

    else:
        return False

df["Check_5"] = df.apply(lambda x: check_5(address = x.Addr_CS, 
                          distance = x.Distance,
                          name = x.Name_CS,
                          streetnumber = x.StrNum_Match), axis=1)

In [6]:
# remove duplicates

rows = df.index[(df['Check_1'] == True) | (df['Check_2'] == True) | (df['Check_3'] == True) | (df['Check_4'] == True) | (df['Check_5'] == True)].tolist()
false_rows = df.index[(df['Check_1'] == False) & (df['Check_2'] == False) & (df['Check_3'] == False) & (df['Check_4'] == False) & (df['Check_5'] == False)].tolist()
print(rows)

df_true = df.loc[rows]
df_false = df.loc[false_rows]

df_true.to_csv('outputs/duplicates_CSD.csv', index=False)
df_false.to_csv('outputs/NOT_duplicates_CSD.csv', index=False)
print(df_true)

hashes = df_true #['to_remove'].tolist()
#print(hashes)

final = final [~final['idx'].isin(hashes)]
final.to_csv('deduplicated_CSD.csv', index=False)
final.to_csv('inputs/deduplicated_CSD.csv', index=False)



[52, 124, 487, 496, 526, 549, 551, 555, 563, 577, 582, 627, 630, 670, 679, 684, 686, 693, 697, 699, 704, 735, 747, 774, 779, 783, 820, 821, 852, 855, 856, 913, 920, 941, 943, 976, 981, 985, 988, 991, 1001, 1017, 1030, 1037, 1038, 1042, 1051, 1055, 1056, 1057, 1058, 1062, 1063, 1065, 1066, 1068, 1100, 1107, 1108, 1112, 1162, 1164, 1245, 1292, 1306, 1307, 1311, 1317, 1319, 1320, 1328, 1329, 1333, 1343, 1347, 1370, 1376, 1381, 1383, 1384, 1385, 1386, 1392, 1393, 1443, 1444, 1446, 1452, 1453, 1463, 1464, 1468, 1476, 1533, 1572, 1667, 1678, 1680, 1686, 1687, 1698, 1732, 1752, 1764, 1766, 1768, 1772, 1779, 1781, 1789, 1797, 1798, 1807, 1815, 1902, 1909, 1913, 1914, 1917, 1967, 1975, 1980, 2010, 2012, 2015, 2032, 2142, 2171, 2172, 2179, 2182, 2307, 2345, 2353, 2354, 2355, 2359, 2491, 2500, 2502, 2504, 2506, 2508, 2519, 2525, 2540, 2541, 2544, 2545, 2546, 2550, 2556, 2565, 2566, 2571, 2593, 2636, 2701, 2796, 2832, 2835, 2839, 2840, 2846, 2847, 2849, 2856, 2898, 2901, 2904, 2905, 2909, 2917, 29